In [1]:
!pip install ninja
!pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html



 

     |████████████████████████████████| 108 kB 30.2 MB/s 
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     |██████████████▋                 | 834.1 MB 1.3 MB/s eta 0:12:36tcmalloc: large alloc 1147494400 bytes == 0x55651f030000 @  0x7fc6dea1a615 0x5564e567a3bc 0x5564e575b18a 0x5564e567d1cd 0x5564e576fb3d 0x5564e56f1458 0x5564e56ec02f 0x5564e567eaba 0x5564e56f12c0 0x5564e56ec02f 0x5564e567eaba 0x5564e56edcd4 0x5564e5770986 0x5564e56ed350 0x5564e5770986 0x5564e56ed350 0x5564e5770986 0x5564e56ed350 0x5564e567ef19 0x5564e56c2a79 0x5564e567db32 0x5564e56f11dd 0x5564e56ec02f 0x5564e567eaba 0x5564e56edcd4 0x5564e56ec02f 0x5564e567eaba 0x5564e56eceae 0x5564e567e9da 0x5564e56ed108 0x5564e56ec02f
     |██████████████████▌             | 1055.7 MB 1.5 MB/s eta 0:08:48tcmalloc: large alloc 1434370048 bytes == 0x556563686000 @  0x7fc6dea1a615 0x5564e567a3bc 0x5564e575b18a 0x5564e567d1cd 0x5564e576fb3d 0x5564e56f1458 0x5564e56ec02f 0x5564e567eaba 0x5564e56f12c0 0x5564e5

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os
from argparse import Namespace

import numpy as np
import torch
import sys
import dlib
sys.path.append(".")
sys.path.append("..")
sys.path.append('/content/drive/MyDrive/PSPNet')

from options.test_options import TestOptions
from models.psp import pSp
import cv2
from scripts.align_all_parallel import align_face
import torchvision.transforms as transforms
from moviepy.editor import ImageSequenceClip

#test_opts = TestOptions().parse()

test_opts = Namespace(checkpoint_path=None, couple_outputs=False, data_path='gt_images', exp_dir=None, latent_mask=None, mix_alpha=None, n_images=None, n_outputs_to_generate=5, resize_factors=None, resize_outputs=False, test_batch_size=2, test_workers=2)


test_opts.checkpoint_path = '/content/drive/MyDrive/PSPNet/pretrained_models/psp_ffhq_encode.pt'


# update test options with options used during training
ckpt = torch.load(test_opts.checkpoint_path, map_location='cpu')
opts = ckpt['opts']
opts.update(vars(test_opts))
opts['output_size'] = 1024

opts = Namespace(**opts)



net = pSp(opts)
net.eval()
net.cuda()
#Prepare the PSPNet + StyleGAN network

transform = transforms.Compose([
	transforms.Resize(size=(256, 256)),
	transforms.ToTensor(),
	transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])]
)
#Preprocessing operations for each image

predictor = dlib.shape_predictor("/content/drive/MyDrive/PSPNet/shape_predictor_68_face_landmarks.dat")
#Predict facial landmarks

img1 = align_face(filepath="/content/drive/MyDrive/PSPNet/nuri.jpg", predictor=predictor)
img1 = img1.convert("RGB")

img2 = align_face(filepath="/content/drive/MyDrive/PSPNet/burak.jpg", predictor=predictor)
img2 = img2.convert("RGB")
# Align face using the landmarks

from_im_img1 = transform(img1).unsqueeze(0)
from_im_img2 = transform(img2).unsqueeze(0)
# Do preprocessing on the aligned face

with torch.no_grad():

	input_img1 = from_im_img1.float().cuda()
	_, latent_vector_img1 = net(input_img1, randomize_noise=False, resize=False, return_latents=True)
	# Obtain features from input image

	input_img2 = from_im_img2.float().cuda()
	_, latent_vector_img2 = net(input_img2, randomize_noise=False, resize=False, return_latents=True)
	# Obtain features from input image

	frames = []
	for i in range(10):
  
		latent_vector = latent_vector_img1*(10 - i)/10 + latent_vector_img2*i/10

		result, _ = net.decoder([latent_vector.float()],input_is_latent=True, randomize_noise=False, return_latents=False)
		# Feed features to the StyleGAN

		result = result.squeeze().permute((1,2,0)).cpu().numpy()

		result[result>1] = 1
		result[result<-1] = -1
		result = (255*(result+1)//2).astype(np.uint8)
		frames.append(result)
	
	video = ImageSequenceClip(frames, fps=10)
	video.write_videofile('/content/drive/MyDrive/PSPNet/output.mp4', codec="libx264")


Loading pSp from checkpoint: /content/drive/MyDrive/PSPNet/pretrained_models/psp_ffhq_encode.pt
[MoviePy] >>>> Building video /content/drive/MyDrive/PSPNet/output.mp4
[MoviePy] Writing video /content/drive/MyDrive/PSPNet/output.mp4


100%|██████████| 21/21 [00:00<00:00, 44.02it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/MyDrive/PSPNet/output.mp4 

